<a href="https://colab.research.google.com/github/doxenix/WUM/blob/main/s24216_WUM_lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wprowadzenie do Uczenia Maszynowego

W tym ćwiczeniu poznamy podstawowe metody używane do regresji i klasyfikacji. Zwykle korzystamy z metod wysokiego poziomu do wykonywania tych obliczeń, ale co tak naprawdę one robią w tle?

Do wykonywania tych obliczeń warto używać sensownej bibliteki matematycznej wydajnie implementującą obliczenia takie jak mnożenie i sumowanie macierzy. My w tym celu użyjemy biblioteki Tensorflow, która została stworzona przede wszystkim po to żeby usprawnić proces projektowania głębokich sieci neuronowych.

Zacznijmy od zaimportowania numpy i Tensorflow:

In [1]:
%pylab inline
import tensorflow as tf
import numpy as np


Populating the interactive namespace from numpy and matplotlib


## Tajna funkcja

Do tego ćwiczenia wymyślimy sobie jakąś prostą funkcję liniową:

\begin{equation}
y = x \cdot 10 - 7
\end{equation}

Nikomu nie powiemy co to jest za funkcja, ale wyliczmy na jej podstawie  przykładowe wartości.

Zaimplementuj powyższą funkcję w pythonie:

In [2]:
func  = lambda x: x * 10 - 7
  

Teraz stwórz jednowymiarową macierz 100 rzeczywistych liczb losowych i zapisz ją pod nazwą `data_X`. Potem użyj tych wartości do wyliczenia odpowiadających im wartości powyższej funkcji i zapisz w zmiennej `data_y`.

In [3]:

data_X = tf.constant(np.linspace(0,2,100), dtype=tf.float32)

'''
Na początku, data_X zapisywałem do np.linspace(0,2,100) bez obiektu tf.constant. Jedank, gdy poźniej nadpisywałem ją i dodowalem obiekt tf.constant, 
przy wyliczaniu gradientu i funkcji straty otrzymywałem dziwne wyniki (funkcja strata "skakała") Jednak gdy od razu zbudowałem obiekt tf.constant i w nim wygenerowałem 100 wartości, problem zniknał)
'''

data_y = func(data_X)
print(data_y)
print(data_X)


tf.Tensor(
[-7.         -6.79798    -6.5959597  -6.3939395  -6.1919193  -5.989899
 -5.787879   -5.5858583  -5.3838387  -5.181818   -4.9797983  -4.7777777
 -4.5757575  -4.3737373  -4.171717   -3.969697   -3.7676768  -3.5656567
 -3.3636363  -3.161616   -2.9595962  -2.7575755  -2.5555553  -2.3535357
 -2.151515   -1.9494953  -1.7474747  -1.5454545  -1.3434343  -1.1414142
 -0.939394   -0.7373738  -0.53535366 -0.33333302 -0.13131332  0.07070732
  0.2727275   0.47474718  0.6767678   0.878788    1.0808077   1.2828283
  1.484849    1.6868687   1.8888893   2.090909    2.2929287   2.4949493
  2.69697     2.8989897   3.1010094   3.30303     3.5050507   3.7070713
  3.909091    4.1111116   4.3131313   4.515151    4.7171717   4.9191923
  5.121212    5.3232327   5.5252523   5.727272    5.9292927   6.1313133
  6.333334    6.5353527   6.7373734   6.939394    7.1414146   7.3434343
  7.545455    7.7474747   7.9494944   8.151515    8.353536    8.555555
  8.757576    8.959596    9.161615    9.363636    9.56

Celem całego ćwiczenia będzie odtworzenie tej funkcji na podstawie przykładowych danych jakie wygenerowaliśmy. Do tego użyjemy "modelu", który będzie odpowiednio odzwierciedlał tą funkcję:

\begin{equation}
y = a \cdot x + b
\end{equation}

Czyli naszym zadaniem będzie odgadnięcie parametrów `a` i `b` z powyższego wzoru, które minimalizują błąd na przykładowych danych. Można to zrobić "strzlając" losowo aż nam się nie uda, ale dużo efektywniej będzie stosowanie jakiegoś algorytmu optymalizującego, np SGD.

## Pierwsze obliczenia w TF

Jeśli chcemy wyliczyć tą samą funkcję co wyżej, ale używając Tensorflow, musimy "opakować" zmienne w objekty typu `tf.Tensor`. Do stworzenia tych objektów używamy typów `tf.constant` i `tf.Variable`. Dla naszych ćwiczeń, dane wygenerowane wyżej będziemy traktować jako stałe, a parametry które chcemy wyliczyć (czyli wartości `a` i `b` tajnego wzoru) będziemy traktować jako zmienne.

Stwórz stałą dla macierzy `data_X` i nazwij ją `X`, a potem stwórz również zmienne `a` i `b` i zainicjuj je jakimiś losowymi wartościami rzeczywistymi. Napisz wzór na wyliczenie naszej funkcji:
```
y = a*X + b
```

i wypisz wynik na ekran: `print(y.numpy())`

In [4]:
X = tf.constant(data_X, dtype=tf.float32)
a = tf.Variable([2.], dtype=tf.float32, name='a')
b = tf.Variable([2,], dtype=tf.float32, name='b')
y = a*X + b
print(X)

tf.Tensor(
[0.         0.02020202 0.04040404 0.06060606 0.08080808 0.1010101
 0.12121212 0.14141414 0.16161616 0.18181819 0.2020202  0.22222222
 0.24242425 0.26262626 0.28282827 0.3030303  0.32323232 0.34343433
 0.36363637 0.3838384  0.4040404  0.42424244 0.44444445 0.46464646
 0.4848485  0.5050505  0.5252525  0.54545456 0.56565654 0.5858586
 0.6060606  0.6262626  0.64646465 0.6666667  0.68686867 0.7070707
 0.72727275 0.74747473 0.7676768  0.7878788  0.8080808  0.82828283
 0.8484849  0.86868685 0.8888889  0.90909094 0.9292929  0.94949496
 0.969697   0.989899   1.010101   1.030303   1.050505   1.0707071
 1.0909091  1.1111112  1.1313131  1.1515151  1.1717172  1.1919192
 1.2121212  1.2323233  1.2525252  1.2727273  1.2929293  1.3131313
 1.3333334  1.3535353  1.3737373  1.3939394  1.4141414  1.4343435
 1.4545455  1.4747474  1.4949495  1.5151515  1.5353535  1.5555556
 1.5757576  1.5959595  1.6161616  1.6363636  1.6565657  1.6767677
 1.6969697  1.7171717  1.7373737  1.7575758  1.7777778  1.79

## Wyliczenie błędu

Powyższe wyniki są oczywiście inne niż te, które mamy wyliczone za pomocą naszej tajnej funkcji. Możemy to oszacować wyliczając różnice między tym co nasz model wyliczył, a tym co powinien wyliczyć:

In [5]:
diff = abs(y.numpy() - data_y)
diff

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([9.        , 8.838384  , 8.676767  , 8.515152  , 8.353536  ,
       8.191919  , 8.030303  , 7.8686867 , 7.7070713 , 7.5454545 ,
       7.3838387 , 7.2222223 , 7.060606  , 6.8989897 , 6.737374  ,
       6.5757575 , 6.4141417 , 6.2525253 , 6.090909  , 5.9292927 ,
       5.767677  , 5.6060605 , 5.444444  , 5.2828283 , 5.121212  ,
       4.959596  , 4.79798   , 4.6363635 , 4.4747477 , 4.3131313 ,
       4.151515  , 3.9898992 , 3.8282828 , 3.6666665 , 3.5050507 ,
       3.343434  , 3.181818  , 3.0202022 , 2.8585858 , 2.6969695 ,
       2.535354  , 2.3737373 , 2.2121208 , 2.0505052 , 1.8888884 ,
       1.727273  , 1.5656571 , 1.4040406 , 1.242424  , 1.0808082 ,
       0.9191923 , 0.757576  , 0.59595966, 0.43434286, 0.272727  ,
       0.11111069, 0.05050516, 0.21212101, 0.37373734, 0.53535366,
       0.6969695 , 0.8585863 , 1.0202017 , 1.1818175 , 1.3434343 ,
       1.5050507 , 1.666667  , 1.8282824 , 1.9898987 , 2.151515  ,
       2.31313

Pytanie jest czy to jest najlepsza metoda na wyliczenie błędu? Otóż jedną z najbardziej podstawowych i popularnych funkcji do tego zasotosowania jest tzw. błąd średnio-kwadratowy, albo po angielsku Mean Square Error (MSE). Powód dla którego stosujemy kwadrat błędu jest (m.in.) bardzo wygodna pochodna tej funkcji (o czym się przekonamy za chwilę), ale na wstępie spróbujmy wyliczyć tą liczbę używając TF.

Zacznijmy od dodania stałej dla macierzy `data_y` o nazwie `y_true`. Potem w jednej linii wylicz różnicę między powyższym `y` i nowym `y_true`, wynik podnieś do kwadratu (w Pythonie się używa składni `x**2`) i wylicz średnią z całości metodą `tf.reduce_mean`:

In [6]:
y_true = tf.constant(data_y, dtype=tf.float32)
print(tf.reduce_mean(tf.math.squared_difference(y, y_true)))

tf.Tensor(22.76431, shape=(), dtype=float32)


## Liczenie gradientu

Zależy nam na minimializacji tej funkcji błędu do wartości bardzo bliskiej zera. W tym celu użyjemy wartości pochodnej (tzw. gradientu) tej funkcji względem poszczególnych zmiennych (parametrów `a` i `b`) żeby ją odjąć od wartości tych parametrów (czyli iść w kierunku odwrotnym od gradientu) co nas doprowadzi do minimum tej funkcji.

Na tym etapie moglibyśmy po prostu wyliczyć wzór do pochodnej funkcji kosztu na kartce papieru i zaimplementować go tak jak to zrobiliśmy wyżej, ale ta metoda jest możliwa tylko dla najprostszych modeli i funkcji błędu. Na szczęście, okazuje się, że liczenie gradientu można robić całkowicie automatycznie i algorytmicznie. Jeśli możemy zdefiniować wzór gradientu dla każdego komponentu naszego grafu obliczeń, gradient całej funkcji można łatwo zdefiniować dzięki regule łańcuchowej (ang. chain rule).

W TF jest to realizowane na różne sposoby, w zależności od metody programowania jakiej używamy, ale na tym ćwiczeniu zasotsujemy coś co się nazywa `tf.GradientTape`. Jest to pewien objekt kontekstowy, który "rejestruje" wszystkie obliczenia i pozwala w dowolnym momemncie wyliczyć pochodną danej funkcji.

Dla przykładu zróbmy mały test na funkcji $y=x^2$:

1. stwórz kontekst gradientu poleceniem: `with tf.GradientTape() as g:`
2. w bloku ze wcięciem zacznij od stworzenia stałej `x` z wartością `3.0` (funkcje z gradientem w TF muszą być liczbami rzeczywistymi)
3. użyj gradientu żeby zarejestrować tą stałą poleceniem: `g.watch(x)`
4. wylicz wartość funkcji y: `y=x**2`
5. wypisz wartość `y` na ekran
6. wylicz gradient funkcji `y` w punkcie `x` poleceniem: `d=g.gradient(y,x)`
7. wpisz graident na ekran

Wartość wypisana na podstawie pkt 5 będzie 9.0 (czyli $3^2$), a wartość z pkt 7 będzie 6.0 (czyli $2\cdot3$, ponieważ pochodna funkcji $x^2$ jest $2x$):

In [7]:
with tf.GradientTape() as g:
  x = tf.constant([3.0], dtype=tf.float32)
  g.watch(x)
  y=x**2
  print(y)
  d=g.gradient(y,x)
print(d)

tf.Tensor([9.], shape=(1,), dtype=float32)
tf.Tensor([6.], shape=(1,), dtype=float32)


## Łaczenie wszystkiego w całość

Zacznijmy od przekopiowania komponentów wyżej do jednego bloku. Skopiuj definicję stałych `X` i `y_true` oraz zmiennych `a` i `b`.

Zdefiniuj też wartość `alpha` równą 0.1, która będzie naszym współczynnikiem uczenia.

W nowym kontekście `tf.GradientTape`:
1. wylicz funkcję `y` na podstawie jej wzoru, tak jak wyżej
2. wylicz funkcję błędu MSE, też tak jak wyżej
3. wypisz wartość funkcji błędu na ekran
4. wylicz gradient funkcji błędu względem zmiennych `a` i `b` - zmienne te możesz podać razem w liście, w wyniku otrzymasz taką samą listę wartości gradientu
5. od poszczególnych zmiennych odejmij odpowiednie wartości gradientu pomnożone przez współczynnik uczenia `alpha`

Do wykonania ostatniego kroku użyj funkcji składowej `assign_sub`, czyli przykładowo: `a.assign_sub(grad[0]*alpha)`. 

Wypisz wartości zmiennych `a` i `b` przed i po modyfikacji gradientu oraz wylicz jeszcze raz funkcję błędu (w tym celu trzeba ponownie wyliczyć zarówno funkcję `y` jak i samą funkcję błędu). Czy zmalała?

In [8]:
X = tf.constant(data_X, dtype=tf.float32)
Y = 10 * X - 7
a = tf.Variable([2.], dtype=tf.float32, name='a')
b = tf.Variable([2.], dtype=tf.float32, name='b')

alpha = 0.1   

with tf.GradientTape() as g:
  g.watch(a)
  g.watch(b)
  Y_pred = a*X + b
  loss = tf.reduce_mean(input_tensor=((Y_pred) - Y)**2)
  print(loss)
  d=g.gradient(loss,[a,b])
  a.assign_sub(d[0] * alpha)
  b.assign_sub(d[1] * alpha)
  Y_pred = a*X + b
  loss = tf.reduce_mean(input_tensor=((Y_pred) - Y)**2)
  print(loss)



tf.Tensor(22.76431, shape=(), dtype=float32)
tf.Tensor(21.241243, shape=(), dtype=float32)


## Pętla trenująca

Teraz skopiuj cały ten kod do bloku poniżej, ale usuń z niego ponowne wyliczanie błędu i wypisywanie parametrów `a` i `b`.

Zamiast tego umieść cały kontekst gradientowy w pętli `for` odliczającej numery epok od 0 do 1000. W każdej iteracji pętli wypisz numer epoki, wypisz obecną wartość parametrów `a` i `b` oraz wartość funkcji błędu.

Po uruchomieniu pętli, powinieneś zauważyć, że wartość błędu maleje do 0, a wartośći `a` i `b` zbiegają do tych z samego początku tego ćwiczenia:

In [9]:
X = X
Y = 10 * X - 7
a = tf.Variable([1.], dtype=tf.float32, name='a')
b = tf.Variable([1.], dtype=tf.float32, name='b')

alpha = 0.1   

for epoch in range(1000):
  with tf.GradientTape() as g:
    g.watch(a)
    g.watch(b)
    Y_pred = a*X + b
    loss = tf.reduce_mean(input_tensor=((Y_pred) - Y)**2)
    print(loss)
    d=g.gradient(loss,[a,b])
    a.assign_sub(d[0] * alpha)
    b.assign_sub(d[1] * alpha)
  print(f'Epoch: {epoch}, a: {a}, b: {b}, loss: {loss}')



tf.Tensor(28.545454, shape=(), dtype=float32)
Epoch: 0, a: <tf.Variable 'a:0' shape=(1,) dtype=float32, numpy=array([1.8121212], dtype=float32)>, b: <tf.Variable 'b:0' shape=(1,) dtype=float32, numpy=array([1.2], dtype=float32)>, loss: 28.545454025268555
tf.Tensor(22.798723, shape=(), dtype=float32)
Epoch: 1, a: <tf.Variable 'a:0' shape=(1,) dtype=float32, numpy=array([2.3665829], dtype=float32)>, b: <tf.Variable 'b:0' shape=(1,) dtype=float32, numpy=array([1.1975758], dtype=float32)>, loss: 22.798723220825195
tf.Tensor(20.133677, shape=(), dtype=float32)
Epoch: 2, a: <tf.Variable 'a:0' shape=(1,) dtype=float32, numpy=array([2.7729263], dtype=float32)>, b: <tf.Variable 'b:0' shape=(1,) dtype=float32, numpy=array([1.0847441], dtype=float32)>, loss: 20.133676528930664
tf.Tensor(18.497517, shape=(), dtype=float32)
Epoch: 3, a: <tf.Variable 'a:0' shape=(1,) dtype=float32, numpy=array([3.0929306], dtype=float32)>, b: <tf.Variable 'b:0' shape=(1,) dtype=float32, numpy=array([0.91321003], dty

## Bardziej ambitne zadanie - klasyfikacja wieloklasowa

Zamiast się bawić w proste funkcje i kilka parametrów, weźmy konkretny przykład. Z modułu `sklearn.datasets` użyj funkcji `load_wine()` i zapisz do zmiennej `data`.

Zaimportuj również metody `label_binarize` oraz `scale` z modułu `sklearn.preprocessing`:

In [10]:
import sklearn.datasets
import sklearn.preprocessing as skp

data = sklearn.datasets.load_wine()

Do zmiennej `data_X` wczytaj macierz `data['data']`, ale dodatkowo ją zamień na typ `float32` używając funkcji składowej `.astype()` i całość znormalizuj przepuszczając przez funkcję `scale()`.

Do zmiennej `data_y_lab` skopiuj tabelę `data['target']`, a potem dodatkowo do innej zmiennej `data_y` zapisz tą samą tabelę przetworzoną funkcją `label_binarize`. Jako drugi parametr tej funkcji podaj argument `classes=[0,1,2]`:

In [11]:
data_X = skp.scale(data['data'].astype(float32))
data_y_lab = data['target']
data_y = skp.label_binarize(data_y_lab, classes=[0,1,2])


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:255: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  "Numerical issues were encountered "


Skopiuj pętle trenującą wyżej i wprowadź do niej następujące zmiany:

Zamiast parametru `a` utwórz macierz dwuwymiarową `W` zincjowaną losowymi wartościami o rozmiarze równym ilości próbek z `data_X` (czyli `data_X.shape[1]`) w jednej osi, a wartością 3 (czyli ilością klas wyjściowych) na drugiej osi. Pamiętaj o konwersji tej macierzy liczb losowych na typ `float32` metoda `.astype()`.

Teraz zauważ jak wyliczymy iloczyn `X` o rozmiarze $178\times13$ i `W` o rozmiarze $13\times3$, w wyniku otrzymamy tabelkę o rozmiarze $178\times3$, czyli wartość każdej klasy wyjściowej dla każdej próbki.

Usuń liczenie funkcji `y` i zamiast tego wylicz coś co nazwiemy `logits`. Wzór na liczenie tego użyje funkcji `tf.matmul` czyli: `logits=tf.matmul(X,W)+b`

Teraz wyliczmy funkcję decyzyjną (nazwijmy ją `y_pred`) dla naszego klasyfikatora. W tym celu (w jednej linii) przepuścimy nasze `logits` przez funkcję `tf.nn.softmax()`, a potem przez `tf.argmax`, podając do tej funkcji argument `axis=1` na drugim miejscu.

Żeby wyliczyć accuracy musimy najpierw porównać do siebie `y_pred.numpy()` i `data_y_lab` operatorem `==`, a potem zsumować wynikową liste wartości true/false (zwykłą funkcją `sum`) i podzielić sumę przez ilość próbek (czyli `data_y_lab.size`). Wypisz accuracy na ekran w każdej epoce.

Do wyliczenia funkcji błędu użyjemy czegoś bardziej adekwatnego niż MSE (które sie stosuje częściej do regresji zamiast klasyfikacji), czyli entropii krzyżowej. Nie musimy jej implementować "ręcznie" tylko użyjemy funkcji `tf.softmax_cross_entropy_with_logits` podając jej jako argumenty stałą `y_true` oraz wynik wyliczenia funkcji `logits`. Dodatkowo zsumujemy wynik tej funkcji używając `tf.reduce_sum`.

Liczenie gradientu jest takie same - trzeba tylko pamiętać o zmianie parametru `a` na `W`, zarówno w liczeniu gradientu, jak i modyfikacji tego parametru później.

Jak się wszystko się udało, model ten powinien osiągać 100% accuracy w około 20-30 epok.

In [12]:
W = tf.Variable(np.random.rand(data_X.shape[1], 3),dtype=tf.float32)
X = tf.constant(data_X, dtype=tf.float32)
b = tf.Variable([1.], dtype=tf.float32, name='b')
y_true = tf.constant(data_y, dtype=tf.float32)

for epoch in range(30):
  with tf.GradientTape() as g:
        
    logits=tf.matmul(X,W)+b
    y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
    accuracy = sum(y_pred.numpy() == data_y_lab) / data_y_lab.size
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y_true, logits))      
    d=g.gradient(loss,[W,b])
    W.assign_sub(d[0] * alpha)
    b.assign_sub(d[1] * alpha)
  print(f'Epoch: {epoch}, loss: {loss}, accuracy: {accuracy}')

Epoch: 0, loss: 264.6387023925781, accuracy: 0.23595505617977527
Epoch: 1, loss: 240.18914794921875, accuracy: 0.9044943820224719
Epoch: 2, loss: 102.59046936035156, accuracy: 0.949438202247191
Epoch: 3, loss: 55.56282424926758, accuracy: 0.9662921348314607
Epoch: 4, loss: 30.531999588012695, accuracy: 0.9719101123595506
Epoch: 5, loss: 14.834046363830566, accuracy: 0.9943820224719101
Epoch: 6, loss: 11.398340225219727, accuracy: 0.9943820224719101
Epoch: 7, loss: 9.217607498168945, accuracy: 0.9943820224719101
Epoch: 8, loss: 7.226371765136719, accuracy: 0.9943820224719101
Epoch: 9, loss: 5.300699710845947, accuracy: 0.9943820224719101
Epoch: 10, loss: 3.4596972465515137, accuracy: 0.9943820224719101
Epoch: 11, loss: 1.921595811843872, accuracy: 0.9943820224719101
Epoch: 12, loss: 1.0947812795639038, accuracy: 1.0
Epoch: 13, loss: 0.7946348190307617, accuracy: 1.0
Epoch: 14, loss: 0.6500074863433838, accuracy: 1.0
Epoch: 15, loss: 0.5616942644119263, accuracy: 1.0
Epoch: 16, loss: 0.5

# Praca domowa - MLP

Jeśli to nie jest oczywiste, powyższy kod jest wstępem do zrobienia MLP. Na pracę domową rzoszerz powyższy model o dodatkową wartstwę ukrytą perceptrona wielowarstwowego.

W tym celu dodaj kolejną zmienną (podobną do `W`) o nazwie `H` (jako hidden) o rozmiarze $178\times10$ (gdzie 10 to ilość jednostek ukrytych - to można zmienić na dowolną wartość) i bias warstwy ukrytej o nazwie `hb`. Zmień też rozmiar zmiennej W na $10\times3$ żeby odzwierciedlić rozmiar nowej warstwy ukrytej.

Przed wyliczeniem `logits` wylicz napierw do zmiennej `hidact` iloczyn `X` i `H` (z biasem `hb`), a potem je przepuszcz przez funkcję aktywacji, np. `tf.sigmoid`. Wtedy w liczeniu `logits` użyj `hidact` zamiast `X`.

Pamiętaj o rozszezrzeniu gradientu o nowe zmienne.

Nowy model niekoniecznie będzie zbiegał szybciej do 100%, ale powinien osiągnąć 100% w te same 30 epok. Można eksperymentować z różnymi ustawieniami żeby osiągnąć lepszy efekt.

In [13]:
W = tf.Variable(np.random.rand(3, 3),dtype=tf.float32)
H = tf.Variable(np.random.rand(13, 3),dtype=tf.float32)
X = tf.constant(data_X, dtype=tf.float32)
b = tf.Variable([1.], dtype=tf.float32, name='b')
hb = tf.Variable([0.], dtype=tf.float32, name='hb')
y_true = tf.constant(data_y, dtype=tf.float32)

for epoch in range(40):
  with tf.GradientTape() as g:
    hidact = tf.matmul(X,H)+hb
    logits = tf.nn.softmax(tf.matmul(hidact,W) + b)
    y_pred = tf.argmax(tf.nn.softmax(logits), axis=1)
    accuracy = sum(y_pred.numpy() == data_y_lab) / data_y_lab.size
    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(y_true, logits))      
    d=g.gradient(loss,[W,b])
    W.assign_sub(d[0] * alpha)
    b.assign_sub(d[1] * alpha)
  print(f'Epoch: {epoch}, loss: {loss}, accuracy: {accuracy}')

  '''
  Dodanie nowej warstwty, w moim przypadku wpłyneło negatywnie na model.
  Wcześniej, model w jakieś 11-15 epok był w stanie dojść do 100% accuracy.
  Po dodaniu warstwy, funkcja straty wraz z każdą epoką co prawda dalej spada,
  ale model jest w stanie dojść do 70-80% accuracy max i musi przejść przez więcej epok 

  '''

Epoch: 0, loss: 209.76126098632812, accuracy: 0.2696629213483146
Epoch: 1, loss: 162.34341430664062, accuracy: 0.6292134831460674
Epoch: 2, loss: 162.1354217529297, accuracy: 0.6292134831460674
Epoch: 3, loss: 161.9178466796875, accuracy: 0.6404494382022472
Epoch: 4, loss: 161.68606567382812, accuracy: 0.6404494382022472
Epoch: 5, loss: 161.4342803955078, accuracy: 0.6404494382022472
Epoch: 6, loss: 161.1563720703125, accuracy: 0.6404494382022472
Epoch: 7, loss: 160.84820556640625, accuracy: 0.6404494382022472
Epoch: 8, loss: 160.5087432861328, accuracy: 0.6404494382022472
Epoch: 9, loss: 160.1387176513672, accuracy: 0.6460674157303371
Epoch: 10, loss: 159.7406005859375, accuracy: 0.651685393258427
Epoch: 11, loss: 159.31964111328125, accuracy: 0.651685393258427
Epoch: 12, loss: 158.8821258544922, accuracy: 0.6573033707865169
Epoch: 13, loss: 158.43463134765625, accuracy: 0.6573033707865169
Epoch: 14, loss: 157.98362731933594, accuracy: 0.6741573033707865
Epoch: 15, loss: 157.525009155